In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
%%capture
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

In [4]:
%pip install unsloth "xformers==0.0.28.post2" 
%pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "git+https://github.com/unslothai/unsloth.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 61.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━

In [5]:
from datasets import Dataset,load_dataset,DatasetDict,load_from_disk

In [26]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)


==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.48.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [28]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",   
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)


Unsloth: Already have LoRA adapters! We shall skip this step.


In [8]:
dataset = load_dataset("Atharva26/Math-Dial-conversations-dataset",split=None)

README.md:   0%|          | 0.00/352 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2262 [00:00<?, ? examples/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 2262
    })
})

In [10]:
from unsloth.chat_templates import standardize_sharegpt,get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

# dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)


Map:   0%|          | 0/2262 [00:00<?, ? examples/s]

In [11]:
train_test_split = dataset['train'].train_test_split(test_size=0.2)  # 90% train, 10% test
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [14]:
train_dataset['text'][1]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a helpful Mathematics Professor for 7th grade students. Your job is to give hints and help students get to the right answer without revealing the answer directly. Help the student get to the solution for the given question: Steve's new book sells 1,000,000 copies.  He got an advance to pay for 100,000 copies. He gets $2 for each copy of the book sold.  His agent takes 10% of that.  How much money did he keep not counting the money from the advance?<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nSteve sells 1,000,000 - 100,000 = 900,000 copies of the book.\nHe earns $2 x 0.9 = $1.8 for each book sold.\nSo he earned a total of $1.8 x 900,000 = $1,620,000.\nHis agent takes 10% of that, which is $162,000.\nTherefore, Steve kept $1,620,000 - $162,000 = $1,458,000 not counting the money from the advance.\n 1,458,000<|eot_id|><|start_header_id|>assis

In [29]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=160,
        eval_strategy="steps",
        eval_steps=10,
        learning_rate=2e-5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)


Map (num_proc=2):   0%|          | 0/1809 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/453 [00:00<?, ? examples/s]

In [30]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

trainer_stats = trainer.train()


Map:   0%|          | 0/1809 [00:00<?, ? examples/s]

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,809 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 160
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,Validation Loss
10,2.360300,2.414129
20,1.928800,2.111485
30,2.016200,1.987738
40,1.802300,1.933712
50,1.784300,1.886228
60,1.611800,1.850416
70,1.764100,1.825466
80,1.750300,1.806680
90,1.825200,1.792102
100,2.007700,1.781394


In [34]:
model.save_pretrained("llama_3b_math_puzzle_tutor")

In [35]:
tokenizer.save_pretrained("llama_3b_math_puzzle_tutor")

('llama_3b_math_puzzle_tutor/tokenizer_config.json',
 'llama_3b_math_puzzle_tutor/special_tokens_map.json',
 'llama_3b_math_puzzle_tutor/tokenizer.json')

In [33]:
# from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

instruction_prompt = "You are a helpful Mathematics Professor for 7th grade students. Your job is to give hints through other example and help students get to the right answer without revealing the answer directly. Help the student get to the solution for the given question: "

question = """
A rectangle has a length of 12 cm and a width of 8 cm.
If you increase both the length and width by 3 cm, what will be the new area of the rectangle?
Calculate the original area and the new area.
How much does the area increase?"""

instruction =  f"{instruction_prompt}"
messages = [{"role": "system", "content": instruction}]

while True:
    student = input("Student: ")
    if student == "":
        break
    messages.append({"role":"user","content":student})
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    
    outputs = model.generate(input_ids = inputs, max_new_tokens = 512, use_cache = True,
                             temperature = 1.5, min_p = 0.1)
    response=tokenizer.batch_decode(outputs,skip_special_tokens=True)
    # print(response)
    if "assistant" in response[0]:
        assistant_response = response[0].split("assistant")[-1].strip()
    else:
        assistant_response = response[0].strip()

    print("Tutor: ",assistant_response)

Student:  Hi A rectangle has a length of 12 cm and a width of 8 cm.If you increase both the length and width by 3 cm, what will be the new area of the rectangle?Calculate the original area and the new area.How much does the area increase? can  u help me with this


Tutor:  ok, so you know the length and the width of the rectangle already, right? now can we use that to calculate the original area of the rectangle


Student:  originial area is 12*8=96


Tutor:  Hi there please recheck this


Student:  length is 12 and width is 8 so area is 96 


...

Student:  71.825 %


Tutor:  correct


Student:  thanks


Tutor:  you are welcome


Student:  


In [36]:
!zip -r llama_3b_math_puzzle_tutor.zip /kaggle/working/llama_3b_math_puzzle_tutor
from IPython.display import FileLink
FileLink(r'llama_3b_math_puzzle_tutor.zip')

  adding: kaggle/working/llama_3b_math_puzzle_tutor/ (stored 0%)
  adding: kaggle/working/llama_3b_math_puzzle_tutor/README.md (deflated 66%)
  adding: kaggle/working/llama_3b_math_puzzle_tutor/tokenizer.json (deflated 85%)
  adding: kaggle/working/llama_3b_math_puzzle_tutor/special_tokens_map.json (deflated 71%)
  adding: kaggle/working/llama_3b_math_puzzle_tutor/adapter_config.json (deflated 56%)
  adding: kaggle/working/llama_3b_math_puzzle_tutor/adapter_model.safetensors (deflated 7%)
  adding: kaggle/working/llama_3b_math_puzzle_tutor/tokenizer_config.json (deflated 94%)


/kaggle/working/llama_3b_math_puzzle_tutor.zip